In [1]:
using Revise

In [2]:
include("modds/DInclude.jl")

DUtils loaded.


Main.TrainUtils

In [3]:
include("index.jl")

┌ Warning: `@pyimport foo` is deprecated in favor of `foo = pyimport("foo")`.
│   caller = _pywrap_pyimport(::PyObject) at PyCall.jl:399
└ @ PyCall /home/sfdgb/.julia/packages/PyCall/zqDXB/src/PyCall.jl:399


edmonds (generic function with 1 method)

In [4]:
using Statistics: mean
using Base.Iterators: cycle
using Knet: Knet, AutoGrad, Data, param, param0, mat, RNN, value, nll, adam, minibatch, progress!, converge, Random, TimerOutputs, dir, zeroone, progress, sgd, load, save, gc, Param, KnetArray, gpu, Data, relu, training, dropout
using Base.Iterators: flatten
using IterTools: ncycle, takenth
using Random

In [5]:
using .DUtils: MyData1, DData2, CreateModel, GCGC
using .DataLoad
using .ModelStructs: Chain, Biaff, MLP2
using .TrainUtils: trainresults

In [6]:
struct DOut; end;
struct TestMode; end;

In [7]:
struct BiaffREL4; mlpheadarc; mlpdeparc; mlpheadrel; mlpdeprel; W_arc; b_arc; U_rel; W_relhead; W_reldep; b_rel; dropout; end;

BiaffREL4(rhid::Int, h1::Int, h2::Int, numrels::Int; dropout=0.0) =
    BiaffREL4(MLP2(rhid, h1, dropout=dropout) , MLP2(rhid, h1,dropout=dropout), MLP2(rhid, h2,dropout=dropout), MLP2(rhid, h2,dropout=dropout) , param(h1,h1), param0(h1), param(h2 * numrels, h2), param(numrels, h2), param(numrels, h2), param0(numrels),dropout)

(bi::BiaffREL4)(x) = begin
    tts = size(x,3)
    bbs = size(x,2)
    


    archead = bi.mlpheadarc(x)
    arcdep = bi.mlpdeparc(x[:,:,2:end])
    
    relhead = bi.mlpheadrel(x)
    reldep = bi.mlpdeprel(x[:,:,2:end])
    
    numrels = size(bi.b_rel,1)
    
    h1 = size(archead,1)
    #h1 = size(relhead,1)
    h2 = size(reldep,1)
    
    #Sr1 = reshape(S, tts, (tts-1)*bbs)
    #hms = size(archead,1)
    rms = size(archead,1)
    archeadp = permutedims(reshape(archead, rms, bbs, tts), [3, 1, 2])
    archeadpp = permutedims(reshape(archead, rms, bbs, tts), [3, 2, 1])
    


    archeadppr = reshape(archeadpp, bbs*tts, h1)
    WH = bi.W_arc * arcdep
    WHp = permutedims(reshape(WH, size(WH,1), bbs, tts-1), [1,3,2])
    HWH = Knet.bmm(archeadp, WHp)
    Hb = archeadppr * bi.b_arc
    Hbr = reshape(Hb, tts, 1 , bbs)
    S = HWH .+ Hbr
    
    
    #Smaxi = findmax(S,dims=1)
    
    #maxis = 
    
    #rh1s = size(relhead,1)
    
    #relhead_p = permutedims(reshape(relhead, rh1s, bbs, tts), [3,1,2])
    relhead_r = reshape(relhead, h2, bbs, tts)
    
    #reshape(, h2, tts-1, bbs)
    H_rel_head = hcat([relhead_r[:, ind[3], ind[1]] for ind in findmax(value(S), dims=1)[2]]...)
    #println("size: ", size(H_rel_head))
    H_rel_dep_pr = reshape(permutedims(reshape(reldep, h2, bbs, tts-1), [1,3,2]), h2, (tts-1)*bbs)
    #UxH_dep = bi.U_rel * H_rel_dep_pr
    
    H_relhead_1 = reshape(H_rel_head, h2,1,tts-1,bbs)
    #HUH = Knet.bmm(H_relhead_1,H_rel_dep_pr)
    
    H1_rel = reshape(H_relhead_1, 1, h2, tts-1, bbs)
    #U: h x (h x rel)
    
    #U: (h x rel) x h
    U_hxrel_h = reshape(bi.U_rel, h2 * numrels, h2)
    H2 = reshape(permutedims(reshape(reldep, h2, bbs, tts-1), [1,3,2]), h2, bbs*(tts-1))
    H2_times_U = U_hxrel_h * H2 # result: (h x rel) x (t x b)
    
    H2_times_U_rs = reshape(H2_times_U, h2, numrels, tts-1, bbs)
    
    H1UH2 = Knet.bmm(H1_rel, H2_times_U_rs)
    H1Uh2_rs = reshape(H1UH2, numrels, tts-1, bbs)
    #subres2
    
    #reldep2_1 = permutedims(reshape(reldep, h2, bbs,tts-1),[3,1,2])
    
    reldep3 = reshape(permutedims(reshape(reldep, h2, bbs,tts-1),[1,3,2]), h2, (tts-1)*bbs)
    
    WxREL_dep = bi.W_reldep * reldep3
    WxREL_head = bi.W_relhead * H_rel_dep_pr
    WxREL_plus = reshape(WxREL_dep + WxREL_head, numrels, tts-1, bbs)
    #subres1
    p_b_rel = reshape(bi.b_rel, numrels, 1, 1)
    
    Srel = H1Uh2_rs + WxREL_plus .+ p_b_rel
    
    
    #Srel = rel_1p2 .+ bi.b_rel
    


    
    return (S,Srel)
end

In [8]:
(bi::BiaffREL4)(x, ::TestMode) = begin #parser version
    
    tts = size(x,3)
    bbs = size(x,2)
    


    archead = bi.mlpheadarc(x)
    arcdep = bi.mlpdeparc(x[:,:,2:end])
    
    relhead = bi.mlpheadrel(x)
    reldep = bi.mlpdeprel(x[:,:,2:end])
    
    numrels = size(bi.b_rel,1)
    
    h1 = size(archead,1)
    #h1 = size(relhead,1)
    h2 = size(reldep,1)
    
    #Sr1 = reshape(S, tts, (tts-1)*bbs)
    #hms = size(archead,1)
    rms = size(archead,1)
    archeadp = permutedims(reshape(archead, rms, bbs, tts), [3, 1, 2])
    archeadpp = permutedims(reshape(archead, rms, bbs, tts), [3, 2, 1])
    


    archeadppr = reshape(archeadpp, bbs*tts, h1)
    WH = bi.W_arc * arcdep
    WHp = permutedims(reshape(WH, size(WH,1), bbs, tts-1), [1,3,2])
    HWH = Knet.bmm(archeadp, WHp)
    Hb = archeadppr * bi.b_arc
    Hbr = reshape(Hb, tts, 1 , bbs)
    S = HWH .+ Hbr
    
    #same until this part ===================================================================================
    Svalr = reshape(value(S), tts,tts-1) #2d version , assert(bbs==1)
    Svalrcpu = Array(Svalr)
    S_cpu_cat = hcat(Array(zeros(tts,1)), Svalrcpu)
    rel_indices = [x+1 for x in chu.chuliu_edmonds_one_root(S_cpu_cat')[2:end]]
    
    S_mst = zeros(tts,tts-1,bbs)
    for (i,rel) in enumerate(rel_indices)
        S_mst[rel,i,1] = 1
    end
    
    
    relhead_r = reshape(relhead, h2, bbs, tts)
    H_rel_head = hcat([relhead_r[:, 1, ind] for ind in rel_indices]...)
        
        
    #same after this part ====================================================================================
    
    H_rel_dep_pr = reshape(permutedims(reshape(reldep, h2, bbs, tts-1), [1,3,2]), h2, (tts-1)*bbs)
    #UxH_dep = bi.U_rel * H_rel_dep_pr
    
    H_relhead_1 = reshape(H_rel_head, h2,1,tts-1,bbs)
    #HUH = Knet.bmm(H_relhead_1,H_rel_dep_pr)
    
    H1_rel = reshape(H_relhead_1, 1, h2, tts-1, bbs)
    #U: h x (h x rel)
    
    #U: (h x rel) x h
    U_hxrel_h = reshape(bi.U_rel, h2 * numrels, h2)
    H2 = reshape(permutedims(reshape(reldep, h2, bbs, tts-1), [1,3,2]), h2, bbs*(tts-1))
    H2_times_U = U_hxrel_h * H2 # result: (h x rel) x (t x b)
    
    H2_times_U_rs = reshape(H2_times_U, h2, numrels, tts-1, bbs)
    
    H1UH2 = Knet.bmm(H1_rel, H2_times_U_rs)
    H1Uh2_rs = reshape(H1UH2, numrels, tts-1, bbs)
    #subres2
    
    #reldep2_1 = permutedims(reshape(reldep, h2, bbs,tts-1),[3,1,2])
    
    reldep3 = reshape(permutedims(reshape(reldep, h2, bbs,tts-1),[1,3,2]), h2, (tts-1)*bbs)
    
    WxREL_dep = bi.W_reldep * reldep3
    WxREL_head = bi.W_relhead * H_rel_dep_pr
    WxREL_plus = reshape(WxREL_dep + WxREL_head, numrels, tts-1, bbs)
    #subres1
    p_b_rel = reshape(bi.b_rel, numrels, 1, 1)
    
    Srel = H1Uh2_rs + WxREL_plus .+ p_b_rel
    
    
    #Srel = rel_1p2 .+ bi.b_rel
    


    
    return (S_mst,Srel)
    
    
end

In [9]:
struct ModelA1; embedLayer::ModelStructs.Embed; rnnLayer::RNN; biaffineLayer; end;

In [10]:
ModelA1(wmat, w_embed_s::Int, rnnh::Int, rnn_ls::Int, h1::Int, h2::Int, numrels::Int; dropout=0.0) = ModelA1(ModelStructs.Embed(wmat, KnetArray, ModelStructs.EmbMatPar()), RNN(w_embed_s, rnnh, bidirectional=true, numLayers=rnn_ls, dropout=dropout), BiaffREL4(2*rnnh, h1,h2,numrels, dropout=dropout))


(model::ModelA1)(x) = begin
    (S,Srel) = model.biaffineLayer(model.rnnLayer(model.embedLayer(x)))
end


(model::ModelA1)(x,y) = begin #loss
    (S,Srel) = model(x)
    l1 = nll(S,y[1])
    #println("Srel size: ", size(Srel))
    #println("y[2] size: ", size(y[2]))
    l2 = nll(Srel,y[2])
    return l1 + l2
end

In [11]:
(model::ModelA1)(x,testtest::TestMode) = begin #parser version
    
    afterRNN = model.rnnLayer(model.embedLayer(x))
    Slast = model.biaffineLayer(afterRNN, testtest)
    
end

In [12]:
struct EmbMatT end
struct EmbMatPar end

struct Embed; w; end

Embed(vocab::Int,embed::Int)=Embed(param(embed,vocab))
Embed(wmat, ::EmbMatT) = Embed(wmat)
Embed(wmat, aType, ::EmbMatPar) = Embed(param(wmat, atype = aType))
Embed(wmat, ::EmbMatPar) = Embed(param(wmat))

(e::Embed)(x) = e.w[:,x]  # (B,T)->(X,B,T)->rnn->(H,B,T)

In [13]:
struct ModelA1POS4; uEmbedLayer::Embed; xEmbedLayer::Embed; embedLayer::Embed; rnnLayer::RNN; biaffineLayer; end;

In [259]:
ModelA1POS4(pos_s, wmat, w_embed_s::Int, rnnh::Int, rnn_ls::Int, h1::Int, h2::Int, numrels::Int, num_upos, num_xpos; dropout=0.0) = ModelA1POS4(Embed(num_upos, pos_s), Embed(num_xpos, pos_s),Embed(wmat, KnetArray, EmbMatPar()), RNN(w_embed_s + pos_s, rnnh, bidirectional=true, numLayers=rnn_ls, dropout=dropout), BiaffREL4(2*rnnh, h1,h2,numrels, dropout=dropout))


(model::ModelA1POS4)(x) = begin
    
    d_out= 0.33
    #println(typeof(x[2]))
    #ax2 = KnetArray{Int64,2}(toBin(Array(x[2]),d_out))
    #ax3 = KnetArray{Int64,2}(toBin(Array(x[3]),d_out))
    ax2 = toBin(x[2],d_out)
    ax3 = toBin(x[3],d_out)
    
    u_embed = model.uEmbedLayer(ax2)
    #println(ax2)
    #println(size(u_embed))
    x_embed = model.xEmbedLayer(ax3)
    w_embed = model.embedLayer(x[1])
    intoRNN = vcat(u_embed + x_embed, w_embed)
    #println(size(intoRNN))
    
    GC.gc()
    (S,Srel) = model.biaffineLayer(model.rnnLayer(intoRNN))
end


(model::ModelA1POS4)(x,y) = begin #loss
    (S,Srel) = model(x)
    l1 = nll(S,y[1])
    #println("Srel size: ", size(Srel))
    #println("y[2] size: ", size(y[2]))
    l2 = nll(Srel,y[2])
    return l1 + l2
end

In [255]:
modelpos(datmb2[1][1])

(K32(53,52,25)[0.833978⋯], K32(37,52,25)[0.791141⋯])

In [239]:
toBin(arr, thold) = [rand()>=thold ? x : 1 for x in arr]

toBin (generic function with 1 method)

In [268]:
a = randn(3,2,3)

3×2×3 Array{Float64,3}:
[:, :, 1] =
  1.08285     1.24525 
  1.22814    -0.183841
 -0.0700382  -0.999403

[:, :, 2] =
  0.504762  0.320515
 -0.414474  0.544226
 -0.613727  0.304184

[:, :, 3] =
 0.112696  1.56468 
 0.194055  0.425302
 0.46698   1.59643 

In [270]:
ka = KnetArray{Float32,2}(a[:,1,:])

3×3 KnetArray{Float32,2}:
  1.08285     0.504762  0.112696
  1.22814    -0.414474  0.194055
 -0.0700382  -0.613727  0.46698 

In [ ]:
toBin(arr, thold) = [rand()>=thold ? x : 1 for x in arr]

In [271]:
dMask(arr, thold) = [rand()>=thold ? 1.0 : 0.0 for x in arr]

dMask (generic function with 1 method)

In [ ]:
rolltwice(thold) = begin
    t1 = rand() >= thold
    t2 = rand() >= thold
    if t1 && !t2
        return (2.0,0.0)
    elseif !t1 && t2
        return (0.0,2.0)
    elseif 
end

In [179]:
@doc rand()

```
rand([rng=GLOBAL_RNG], [S], [dims...])
```

Pick a random element or array of random elements from the set of values specified by `S`; `S` can be

  * an indexable collection (for example `1:9` or `('x', "y", :z)`),
  * an `AbstractDict` or `AbstractSet` object,
  * a string (considered as a collection of characters), or
  * a type: the set of values to pick from is then equivalent to `typemin(S):typemax(S)` for integers (this is not applicable to [`BigInt`](@ref)), to $[0, 1)$ for floating point numbers and to $[0, 1)+i[0, 1)]$ for complex floating point numbers;

`S` defaults to [`Float64`](@ref).

!!! compat "Julia 1.1"
    Support for `S` as a tuple requires at least Julia 1.1.


# Examples

```julia-repl
julia> rand(Int, 2)
2-element Array{Int64,1}:
 1339893410598768192
 1575814717733606317

julia> using Random

julia> rand(MersenneTwister(0), Dict(1=>2, 3=>4))
1=>2
```

!!! note
    The complexity of `rand(rng, s::Union{AbstractDict,AbstractSet})` is linear in the length of `s`, unless an optimized method with constant complexity is available, which is the case for `Dict`, `Set` and `BitSet`. For more than a few calls, use `rand(rng, collect(s))` instead, or either `rand(rng, Dict(s))` or `rand(rng, Set(s))` as appropriate.



In [231]:
asas = KnetArray{Float32,2}(rand(3,4))(100, 25, 53)

3×4 KnetArray{Float32,2}:
 0.873477  0.872114  0.627887  0.346778 
 0.387194  0.782358  0.191152  0.0485165
 0.619192  0.842981  0.94319   0.0514271

In [233]:
toBin(asas,0.33)(100, 25, 53)

MethodError: SYSTEM: show(lasterr) caused an error

In [160]:
rand(3)

3-element Array{Float64,1}:
 0.620617753311258 
 0.4575169961271557
 0.8775783149361016

In [15]:
(model::ModelA1POS4)(x,testtest::TestMode) = begin #parser version
    u_embed = model.uEmbedLayer(x[2])
    x_embed = model.xEmbedLayer(x[3])
    w_embed = model.embedLayer(x[1])
    intoRNN = vcat(u_embed + x_embed, w_embed)
    afterRNN = model.rnnLayer(intoRNN)
    Slast = model.biaffineLayer(afterRNN, testtest)
    
end

In [16]:
function my_accuracy(model, data; dims=1, average=true, o...)
    sum = cnt = 0
    for (x,y) in data
        (z,n) = Knet.accuracy(model(x; o...)[1], y[1]; dims=dims, average=false)
        sum += z
        cnt += n
    end
    average ? sum / cnt : (sum, cnt)
end

my_accuracy (generic function with 1 method)

In [17]:
function my_accuracy2(model, data; dims=1, average=true, o...)
    sum = cnt = 0
    for (x,y) in data
        (z,n) = Knet.accuracy(model(x; o...)[2], y[2]; dims=dims, average=false)
        sum += z
        cnt += n
    end
    average ? sum / cnt : (sum, cnt)
end

my_accuracy2 (generic function with 1 method)

In [18]:
function my_findindices(y,a::AbstractArray{<:Integer}; dims=1)
    ninstances = length(a)
    nindices = 0
    indices = Vector{Int}(undef,ninstances)
    if dims == 1                   # instances in first dimension
        y1 = size(y,1)
        y2 = div(length(y),y1)
        if ninstances != y2; throw(DimensionMismatch()); end
        @inbounds for j=1:ninstances
            #if a[j] == 0; continue; end
            indices[nindices+=1] = (j-1)*y1 + a[j]
        end
    elseif dims == 2               # instances in last dimension
        y2 = size(y,ndims(y))
        y1 = div(length(y),y2)
        if ninstances != y1; throw(DimensionMismatch()); end
        @inbounds for j=1:ninstances
            #if a[j] == 0; continue; end
            indices[nindices+=1] = (a[j]-1)*y1 + j
        end
    else
        error("findindices only supports dims = 1 or 2")
    end
    #println(nindices == ninstances)
    return (nindices == ninstances ? indices : view(indices,1:nindices))
end

my_findindices (generic function with 1 method)

In [19]:
function my_accuracy_rel(model, data; dims=1, average=true, o...)
    sum = cnt = 0
    #ii = 1
    for (x,y) in data
        #println(ii)
        #ii += 1

        (z,n) = my_accuracy_ya(model(x; o...), y; dims=dims, average=false)
        sum += z
        cnt += n
    end
    average ? sum / cnt : (sum, cnt)
end

my_accuracy_rel (generic function with 1 method)

In [20]:
function my_accuracy_rel_parsed(model, data; dims=1, average=true, o...)
    sum = cnt = 0
    #ii = 1
    for (x,y) in data
        #println(ii)
        #ii += 1

        (z,n) = my_accuracy_ya(model(x, TestMode(); o...), y; dims=dims, average=false)
        sum += z
        cnt += n
    end
    average ? sum / cnt : (sum, cnt)
end

my_accuracy_rel_parsed (generic function with 1 method)

In [21]:
function my_accuracy_ya(y,a; dims=1, average=true)
    #println("I'm here!")
    #println(typeof(y[1]))
    y_arc = y[1]
    y_rel = y[2]
    a_arc = a[1]
    a_rel = a[2]
    
    indices = my_findindices(y_arc,a_arc,dims=dims)
    ycpu = convert(Array,y_arc)
    (maxval,maxind) = findmax(ycpu,dims=dims)
    maxind = LinearIndices(ycpu)[maxind]
    maxind = vec(maxind)#[vec(a_arc) .!= 0]
    #println("c1: ", size(indices))
    correct1 = (maxind .== indices)
    
    
    indices = my_findindices(y_rel,a_rel,dims=dims)
    ycpu = convert(Array,y_rel)
    (maxval,maxind) = findmax(ycpu,dims=dims)
    maxind = LinearIndices(ycpu)[maxind]
    maxind = vec(maxind)#[vec(a_rel) .!= 0]
    #println("c1: ", size(indices))
    correct2 = (maxind .== indices)
    
        
    
    correct = correct1 .* correct2
    
    average ? (sum(correct) / length(correct)) : (sum(correct), length(correct))
end

my_accuracy_ya (generic function with 1 method)

In [256]:
function minibREL(data4c,batchsize)
    datdat = sort(data4c, by=x->length(x[1]), rev=true)
    lentsdatdat = DataLoad.lents(datdat)
    uniqdat = unique(lentsdatdat)
    lennnum = [(i,count(x->x==i,lentsdatdat),j) for (j,i) in enumerate(uniqdat)]
    i=1
    j=0
    k=0
    batdat = []
    for (a,b,c) in lennnum
        bb = b
        j += b
        j = i+b
        k=j
        while bb >= batchsize
            j =i+batchsize-1

            push!(batdat, (cat([x[1] for x in datdat[i:j]]...,dims=1), (cat([x[2][1] for x in datdat[i:j]]...,dims=2), Array{Int64,2}(cat([x[2][2] for x in datdat[i:j]]...,dims=2)))))
            i=j+1
            bb -= batchsize
        end
        i += bb
    end
    batdat
end

minibREL (generic function with 1 method)

In [23]:
lents2(datdat) = map(x->length(x[1][1]), datdat)


lents2 (generic function with 1 method)

In [24]:
function minibRELPOS(data4c,batchsize)
    datdat = sort(data4c, by=x->length(x[1][1]), rev=true)
    lentsdatdat = lents2(datdat)
    uniqdat = unique(lentsdatdat)
    lennnum = [(i,count(x->x==i,lentsdatdat),j) for (j,i) in enumerate(uniqdat)]
    i=1
    j=0
    k=0
    batdat = []
    for (a,b,c) in lennnum
        bb = b
        j += b
        j = i+b
        k=j
        while bb >= batchsize
            j =i+batchsize-1

            push!(batdat, (  (  cat([x[1][1] for x in datdat[i:j]]...,dims=1) 
                        , cat([x[1][2] for x in datdat[i:j]]...,dims=1)  
                        , cat([x[1][3] for x in datdat[i:j]]...,dims=1)            )
                    
                    , (cat([x[2][1] for x in datdat[i:j]]...,dims=2), Array{Int64,2}(cat([x[2][2] for x in datdat[i:j]]...,dims=2)))))
            i=j+1
            bb -= batchsize
        end
        i += bb
    end
    batdat
end

minibRELPOS (generic function with 1 method)

In [25]:
wembed, wembedind = DataLoad.load_embed("glove.6B.100d.txt");

In [26]:
wembedmat = DataLoad.get_wembedmat("glove.6B.100d.txt")

100×400001 Array{Float32,2}:
 -0.038194  -0.10767   -0.33979   …  -0.10461    0.28365    0.28365 
 -0.24487    0.11053    0.20941      -0.5047    -0.6263    -0.6263  
  0.72812    0.59812    0.46348      -0.49331   -0.44351   -0.44351 
 -0.39961   -0.54361   -0.64792       0.13516    0.2177     0.2177  
  0.083172   0.67396   -0.38377      -0.36371   -0.087421  -0.087421
  0.043953   0.10663    0.038034  …  -0.4475    -0.17062   -0.17062 
 -0.39141    0.038867   0.17127       0.18429    0.29266    0.29266 
  0.3344     0.35481    0.15978      -0.05651   -0.024899  -0.024899
 -0.57545    0.06351    0.46619       0.40474    0.26414    0.26414 
  0.087459  -0.094189  -0.019169     -0.72583   -0.17023   -0.17023 
  0.28787    0.15786    0.41479   …   0.31079    0.25817    0.25817 
 -0.06731   -0.81665   -0.34349      -0.31763    0.097484   0.097484
  0.30906    0.14172    0.26872      -0.019824  -0.33103   -0.33103 
  ⋮                               ⋱                         ⋮       
  0.3

In [346]:
io = open("en_lines-udpipe.conllu", "r");

In [347]:
sentences = []
words = []
for (i, line) in enumerate(eachline(io))
    if line == ""
        push!(sentences, words)
        words = []
        continue
    end
    if line[1] == '#'
        #println("comment")
        continue
    end
    push!(words, split(line))
end

In [294]:
UPOS_D = Dict()
XPOS_D = Dict()
UPOS_D["UNK"] = 1
XPOS_D["UNK"] = 1
u_ind = 2
x_ind = 2
for words in sentences
    for word in words
        if get(UPOS_D, word[4], -1) == -1
            UPOS_D[word[4]] = u_ind
            u_ind += 1
        end
        if get(XPOS_D, word[5], -1) == -1
            XPOS_D[word[5]] = x_ind
            x_ind += 1
        end
    end
end
UPOS_D["ROOOT"] = u_ind
XPOS_D["ROOOT"] = x_ind

MethodError: MethodError: no method matching *(::Int64, ::typeof(my_accuracy_rel_parsed))
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:529
  *(!Matched::PyObject, ::Any) at /home/sfdgb/.julia/packages/PyCall/zqDXB/src/pyoperators.jl:13
  *(::T, !Matched::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} at int.jl:54
  ...

52

In [295]:
UPOS_D = Dict(
"DET"   => 7,
  "PROPN" => 2,
  "PUNCT" => 3,
  "ADJ"   => 4,
  "SCONJ" => 12,
  "NOUN"  => 5,
  "NUM"   => 13,
  "X"     => 16,
  "VERB"  => 6,
  "ROOOT" => 19,
  "ADV"   => 14,
  "AUX"   => 9,
  "PRON"  => 10,
  "PART"  => 11,
  "CCONJ" => 15,
  "ADP"   => 8,
  "SYM"   => 18,
  "INTJ"  => 17,
  "UNK"   => 1)

Dict{String,Int64} with 19 entries:
  "DET"   => 7
  "PROPN" => 2
  "PUNCT" => 3
  "ADJ"   => 4
  "SCONJ" => 12
  "NOUN"  => 5
  "NUM"   => 13
  "X"     => 16
  "VERB"  => 6
  "ROOOT" => 19
  "ADV"   => 14
  "AUX"   => 9
  "PRON"  => 10
  "PART"  => 11
  "CCONJ" => 15
  "ADP"   => 8
  "SYM"   => 18
  "INTJ"  => 17
  "UNK"   => 1

In [296]:
XPOS_D = Dict{Any,Any}("CC" => 28,"." => 12,"PDT" => 33,"VBP" => 26,"ROOOT" => 52,"VBD" => 7,"WRB" => 35,"VBG" => 16,"NNS" => 6,"NN" => 10,"XX" => 51,"HYPH" => 3,"-LRB-" => 13,":" => 4,"RBS" => 40,"GW" => 50,"VB" => 15,"-RRB-" => 19,"VBZ" => 23,"SYM" => 49,"CD" => 22,"JJS" => 36,"``" => 37,"EX" => 30,"LS" => 44,"," => 8,"NFP" => 48,"FW" => 43,"DT" => 9,"RBR" => 32,"''" => 38,"WDT" => 31,"NNP" => 2,"ADD" => 42,"PRP\$" => 27,"IN" => 11,"WP" => 29,"UNK" => 1,"VBN" => 20,"RB" => 24,"WP\$" => 41,"MD" => 14,"RP" => 21,"JJ" => 5,"\$" => 47,"JJR" => 34,"PRP" => 17,"TO" => 18,"NNPS" => 25,"AFX" => 46,"UH" => 45,"POS" => 39)

Dict{Any,Any} with 52 entries:
  "CC"    => 28
  "."     => 12
  "PDT"   => 33
  "VBP"   => 26
  "ROOOT" => 52
  "VBD"   => 7
  "WRB"   => 35
  "VBG"   => 16
  "NNS"   => 6
  "NN"    => 10
  "XX"    => 51
  "HYPH"  => 3
  "-LRB-" => 13
  ":"     => 4
  "RBS"   => 40
  "GW"    => 50
  "VB"    => 15
  "-RRB-" => 19
  "VBZ"   => 23
  "SYM"   => 49
  "CD"    => 22
  "JJS"   => 36
  "``"    => 37
  "EX"    => 30
  "LS"    => 44
  ⋮       => ⋮

In [297]:
rootupos = UPOS_D["ROOOT"]
rootxpos = XPOS_D["ROOOT"]

52

In [137]:
data2 = DataLoad.load_data3("en-ud-train.conllu", DataLoad.UDEPREL);

_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_
_


In [348]:
forms = [[word[2] for word in words] for words in sentences]#, word[4], word[5]
uposs = [[word[4] for word in words] for words in sentences]
xposs = [[word[5] for word in words] for words in sentences]
heads = [[word[7] for word in words] for words in sentences]
deprels = data2[3]#[[word[8] for word in words] for words in sentences]

12543-element Array{Any,1}:
 Any[1, 34, 22, 34, 5, 28, 33, 30, 22, 22  …  17, 27, 8, 27, 34, 8, 17, 5, 27, 34]
 Any[34, 17, 28, 8, 17, 5, 27, 7, 7, 1, 24, 30, 8, 31, 26, 2, 34, 34]             
 Any[1, 34, 5, 28, 33, 26, 28, 7, 10, 12, 29, 5, 30, 2, 8, 31, 34]                
 Any[28, 8, 27, 7, 7, 1, 8, 29, 31, 8, 17, 27, 8, 17, 27, 34]                     
 Any[17, 28, 8, 27, 14, 1, 8, 17, 12, 31  …  30, 8, 17, 12, 27, 4, 8, 17, 31, 34] 
 Any[17, 28, 7, 7, 1, 8, 17, 31, 8, 17, 12, 27, 34]                               
 Any[28, 1, 26, 28, 7, 10, 17, 30, 8, 27, 12, 31, 34]                             
 Any[17, 28, 7, 1, 8, 17, 12, 9, 12, 34, 13, 31, 8, 17, 27, 34]                   
 Any[26, 17, 28, 7, 4, 3, 26, 37, 30, 26  …  28, 2, 26, 37, 17, 5, 30, 4, 37, 34] 
 Any[28, 1, 30, 4, 4, 8, 17, 12, 31, 28, 7, 4, 2, 8, 17, 5, 31, 8, 31, 34]        
 Any[1, 34, 22, 34, 28, 33, 17, 30, 8, 17, 12, 12, 27, 26, 3, 30, 8, 31, 34]      
 Any[17, 12, 28, 8, 12, 27, 1, 26, 28, 7  …  3, 17, 5, 28, 

In [353]:
#test version
data22 = zip((reshape(x,1,1,length(x)) for x in forms), zip((reshape(x,1,1,length(x)) for x in uposs)), zip((reshape(x,1,1,length(x)) for x in xposs)));

In [354]:
data4 = (   (reshape(cat(DataLoad.rootind,map(DataLoad.getind,x1), dims=3),1,length(x1)+1)  , reshape(cat(rootupos,map(f->get(UPOS_D, f, 1),x2[1]), dims=3),1,length(x2[1])+1)  , reshape(cat(rootxpos,map(f->get(XPOS_D, f, 1),x3[1]), dims=3),1,length(x3[1])+1)        ) for (x1,x2,x3) in data22);

In [355]:
data4c = collect(data4);

In [352]:
data22 = zip( zip((reshape(x,1,1,length(x)) for x in forms), zip((reshape(x,1,1,length(x)) for x in uposs)), zip((reshape(x,1,1,length(x)) for x in xposs)))    ,zip(data2[2],data2[3]));

In [142]:
collect(data22)

DimensionMismatch: DimensionMismatch("dimensions must match")

In [145]:
data4 = (  ( (reshape(cat(DataLoad.rootind,map(DataLoad.getind,x1), dims=3),1,length(x1)+1)  , reshape(cat(rootupos,map(f->get(UPOS_D, f, 1),x2[1]), dims=3),1,length(x2[1])+1)  , reshape(cat(rootxpos,map(f->get(XPOS_D, f, 1),x3[1]), dims=3),1,length(x3[1])+1)        )  ,y) for ((x1,x2,x3),y) in data22);

In [146]:
data4_1 = ((x,(y[1] .+ 1,y[2])) for (x,y) in data4);

In [147]:
data4c = collect(data4_1);


In [120]:
size(datmb2_test[12][1])

(1, 6)

In [122]:
size(sentences[12])

(5,)

In [356]:
datmb2_test = minibRELPOS_test_test(data4c,1);

In [59]:
lents(datdat) = map(x->length(x[1]), datdat)the corresponding word

lents (generic function with 1 method)

In [109]:
datmb2_test[1][2]

1×8 Array{Int64,2}:
 19  10  12  2  2  2  2  3

In [62]:
function minibRELPOS_test_test(data4c,batchsize)
    #datdat = sort(data4c, by=x->length(x[1][1]), rev=true)
    datdat = data4c
    lentsdatdat = lents(datdat)
    uniqdat = unique(lentsdatdat)
    lennnum = [(i,count(x->x==i,lentsdatdat),j) for (j,i) in enumerate(uniqdat)]
    i=1
    j=0
    k=0
    batdat = []
    for (a,b,c) in lennnum
        bb = b
        j += b
        j = i+b
        k=j
        while bb >= batchsize
            j =i+batchsize-1

            push!(batdat,   (  cat([x[1] for x in datdat[i:j]]...,dims=1) 
                        , cat([x[2] for x in datdat[i:j]]...,dims=1)  
                        , cat([x[3] for x in datdat[i:j]]...,dims=1)            )
                    
                    )
            i=j+1
            bb -= batchsize
        end
        i += bb
    end
    batdat
end

minibRELPOS_test_test (generic function with 1 method)

In [148]:
datmb2 = minibRELPOS(data4c,25);

In [149]:
datmb2sh = Knet.shuffle(datmb2);

In [151]:
num_b = 30
num_mb = size(datmb2sh)[1]

470

In [263]:
model = modelpos
opt1 = Knet.Adam()
mparams = Knet.params(model)
for x in mparams
    x.opt = Knet.clone(opt1)
end
println("begin");
for e1 in 1:num_b
    
    println("E1: ", e1);
    println("Train Acc: ", my_accuracy_rel(model,datmb2sh));
    #println("Test Acc: ", my_accuracy_rel(model,dtst))
        for e2 in 1:num_mb
            println("e1: ", e1, ", e2: ", e2)
            y = Knet.@diff model(datmb2sh[e2][1], datmb2sh[e2][2]);
            for x in mparams
                Knet.update!(x, Knet.grad(y,x));
            end
            GC.gc()
            #println(value(y))
        end
    
end
println("Train Acc: ", my_accuracy_rel(model,datmb2sh))
#println("Test Acc: ", my_accuracy_rel(model,dtst))

begin
E1: 1


InterruptException: InterruptException:

In [288]:
model = modelpos
opt1 = Knet.Adam()
mparams = Knet.params(model)
for x in mparams
    x.opt = Knet.clone(opt1)
end
println("begin");
display()
for e1 in 1:11
    
    println("E1: ", e1);
    println("Train Acc: ", my_accuracy_rel(model,datmb2sh));
    #println("Test Acc: ", my_accuracy_rel(model,dtst))
        for e2 in 1:num_mb
            println("e1: ", e1, ", e2: ", e2)
            y = Knet.@diff model(datmb2sh[e2][1], datmb2sh[e2][2]);
            for x in mparams
                Knet.update!(x, Knet.grad(y,x));
            end
            GC.gc()
            #println(value(y))
        end
    
end
println("Train Acc: ", my_accuracy_rel(model,datmb2sh))
#println("Test Acc: ", my_accuracy_rel(model,dtst))

begin
E1: 1
Train Acc: 0.9377215901109707
e1: 1, e2: 1
e1: 1, e2: 2
e1: 1, e2: 3
e1: 1, e2: 4
e1: 1, e2: 5
e1: 1, e2: 6
e1: 1, e2: 7
e1: 1, e2: 8
e1: 1, e2: 9
e1: 1, e2: 10
e1: 1, e2: 11
e1: 1, e2: 12
e1: 1, e2: 13
e1: 1, e2: 14
e1: 1, e2: 15
e1: 1, e2: 16
e1: 1, e2: 17
e1: 1, e2: 18
e1: 1, e2: 19
e1: 1, e2: 20
e1: 1, e2: 21
e1: 1, e2: 22
e1: 1, e2: 23
e1: 1, e2: 24
e1: 1, e2: 25
e1: 1, e2: 26
e1: 1, e2: 27
e1: 1, e2: 28
e1: 1, e2: 29
e1: 1, e2: 30
e1: 1, e2: 31
e1: 1, e2: 32
e1: 1, e2: 33
e1: 1, e2: 34
e1: 1, e2: 35
e1: 1, e2: 36
e1: 1, e2: 37
e1: 1, e2: 38
e1: 1, e2: 39
e1: 1, e2: 40
e1: 1, e2: 41
e1: 1, e2: 42
e1: 1, e2: 43
e1: 1, e2: 44
e1: 1, e2: 45
e1: 1, e2: 46
e1: 1, e2: 47
e1: 1, e2: 48
e1: 1, e2: 49
e1: 1, e2: 50
e1: 1, e2: 51
e1: 1, e2: 52
e1: 1, e2: 53
e1: 1, e2: 54
e1: 1, e2: 55
e1: 1, e2: 56
e1: 1, e2: 57
e1: 1, e2: 58
e1: 1, e2: 59
e1: 1, e2: 60
e1: 1, e2: 61
e1: 1, e2: 62
e1: 1, e2: 63
e1: 1, e2: 64
e1: 1, e2: 65
e1: 1, e2: 66
e1: 1, e2: 67
e1: 1, e2: 68
e1: 1, e2: 69
e

InterruptException: InterruptException:

In [266]:
modelpos = ModelA1POS4(100, wembedmat, 100, 200, 3, 400, 100, 37, rootupos, rootxpos, dropout=0.33)

ModelA1POS4(Embed(P(KnetArray{Float32,2}(100,19))), Embed(P(KnetArray{Float32,2}(100,52))), Embed(P(KnetArray{Float32,2}(100,400001))), LSTM(input=200,hidden=200,bidirectional,layers=3,dropout=0.33), BiaffREL4(Main.ModelStructs.DenseDrop(P(KnetArray{Float32,2}(400,400)), P(KnetArray{Float32,1}(400)), NNlib.relu, 0.33), Main.ModelStructs.DenseDrop(P(KnetArray{Float32,2}(400,400)), P(KnetArray{Float32,1}(400)), NNlib.relu, 0.33), Main.ModelStructs.DenseDrop(P(KnetArray{Float32,2}(100,400)), P(KnetArray{Float32,1}(100)), NNlib.relu, 0.33), Main.ModelStructs.DenseDrop(P(KnetArray{Float32,2}(100,400)), P(KnetArray{Float32,1}(100)), NNlib.relu, 0.33), P(KnetArray{Float32,2}(400,400)), P(KnetArray{Float32,1}(400)), P(KnetArray{Float32,2}(3700,100)), P(KnetArray{Float32,2}(37,100)), P(KnetArray{Float32,2}(37,100)), P(KnetArray{Float32,1}(37)), 0.33))

In [69]:
modelpos = Knet.load("30epochwithdropout_33_1layerPOS.jld2","model")

ModelA1POS4(Embed(P(KnetArray{Float32,2}(100,19))), Embed(P(KnetArray{Float32,2}(100,52))), Embed(P(KnetArray{Float32,2}(100,400001))), LSTM(input=200,hidden=200,bidirectional,dropout=0.33), BiaffREL4(Main.ModelStructs.DenseDrop(P(KnetArray{Float32,2}(400,400)), P(KnetArray{Float32,1}(400)), NNlib.relu, 0.33), Main.ModelStructs.DenseDrop(P(KnetArray{Float32,2}(400,400)), P(KnetArray{Float32,1}(400)), NNlib.relu, 0.33), Main.ModelStructs.DenseDrop(P(KnetArray{Float32,2}(100,400)), P(KnetArray{Float32,1}(100)), NNlib.relu, 0.33), Main.ModelStructs.DenseDrop(P(KnetArray{Float32,2}(100,400)), P(KnetArray{Float32,1}(100)), NNlib.relu, 0.33), P(KnetArray{Float32,2}(400,400)), P(KnetArray{Float32,1}(400)), P(KnetArray{Float32,2}(3700,100)), P(KnetArray{Float32,2}(37,100)), P(KnetArray{Float32,2}(37,100)), P(KnetArray{Float32,1}(37)), 0.33))

In [289]:
Knet.save("27epochwithdropout_33_3layerPOSdrop_fulltrain.jld2","model", modelpos)

In [92]:
my_accuracy(modelpos, datmb2sh)

0.9000337268128162

In [281]:
my_accuracy_rel_parsed(modelpos, datmb2_test)

DimensionMismatch: DimensionMismatch("")

In [95]:
modelpos(datmb2sh[1][1], TestMode())

DimensionMismatch: DimensionMismatch("")

In [357]:
out_sents = deepcopy(sentences);

In [126]:
size(sentences[12],1)

5

In [358]:
senttext = ""
for (i,words) in enumerate(out_sents)
    sent_size = size(words,1)
    #priasr1ntln(i)
    #println(sent_size)
    #println(size(datmb2_test[i][1]))
    arcs = maxs_arc(i, sent_size)
    rels = maxs_rel(i, sent_size)
    
    
    #senttext = ""
    for (j,word) in enumerate(words)
        #wd_size = size(word[1][1])
        #println(wd_size)
        for (k, wd) in enumerate(word)
            if k == 7
                senttext *= string(arcs[j])
            elseif k == 8
                senttext *= rels[j]
            else
                
                senttext *= wd
                
            end
            senttext *= k == size(word,1) ? "\n" : "\t" 
        end
        
        
    end
    senttext *= "\n"
    #print(senttext)
    #if i > 92; break; end
end
#senttext *= "\n";arc

In [359]:
io = open("en_lines-udpipe_27.conllu", "w");

write(io, senttext);

close(io);

In [201]:
size(out_sents[100][1])

(10,)

In [141]:
parsedmaxs(1, 7)

DimensionMismatch: DimensionMismatch("new dimensions (7,) must be consistent with array size 81")

In [152]:
out_sents[1] |> size

(7,)

In [155]:
modelpos(datmb2_test[1][1])[2]

37×7×1 KnetArray{Float32,3}:
[:, :, 1] =
   1.83867   -10.3979   -15.6233   …  -21.2778   -15.3718   -10.8809 
 -20.8331    -17.1477   -27.5157      -18.928    -24.6619   -22.1305 
 -10.807     -19.9217   -16.5187      -21.0032    -8.23008  -16.7919 
  -2.39383    -3.59742  -13.2939       -8.85858   -6.48409  -11.6872 
 -11.7555    -14.8739    -8.55811     -14.5143   -16.9812   -24.3962 
 -11.0127    -27.4971    -6.69582  …  -25.7202   -23.3944    -9.19934
 -24.9457     -4.81204  -19.6008      -24.8261   -27.8925   -15.0902 
 -17.1299     -7.54258  -31.9133       18.1751   -10.4667   -12.6853 
 -17.4439    -18.6662   -31.2323      -22.9009   -43.1272    -8.08623
 -12.4806    -16.611    -28.6762      -25.7911   -17.658    -23.7662 
 -25.062     -32.1519   -43.9773   …  -32.2824   -42.2111   -25.3819 
 -16.0294    -15.1245    -5.63911       1.0784    -8.2204   -12.9964 
  -7.68545   -14.1876   -11.3733      -21.8515   -18.1868   -12.6408 
   ⋮                               ⋱             

In [144]:
modelpos(datmb2_test[1][1])

(K32(82,81,1)[-13.6335⋯], K32(37,81,1)[-8.642⋯])

In [80]:
maxs_arc(ind,sz) = [x[1]-1 for x in reshape(findmax(modelpos(datmb2_test[ind], TestMode())[1], dims=1)[2],sz)]

maxs_arc (generic function with 1 method)

In [115]:
datmb2_test[12][1]

1×6 Array{Int64,2}:
 399999  280  11062  26  400000  806

In [112]:
maxs_arc(1,7) 

7-element Array{Int64,1}:
 0
 1
 6
 6
 6
 1
 1

In [81]:
maxs_rel(ind,sz) = [revdict(x[1]) for x in reshape(findmax(modelpos(datmb2_test[ind], TestMode())[2], dims=1)[2],sz)]

maxs_rel (generic function with 1 method)

In [168]:
maxs_rel(1,7)

7-element Array{String,1}:
 "root" 
 "mark" 
 "nsubj"
 "advcl"
 "case" 
 "obl"  
 "punct"

In [348]:
modelpos(datmb2_test[1][1], TestMode())[1]

8×7×1 Array{Float64,3}:
[:, :, 1] =
 1.0  1.0  1.0  0.0  1.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [163]:
filter((k, v) -> v == 2, DataLoad.UDEPREL)

┌ Warning: In `filter(f, dict)`, `f` is now passed a single pair instead of two arguments.
│   caller = top-level scope at In[163]:1
└ @ Core In[163]:1


Dict{String,Int8} with 1 entry:
  "acl" => 2

In [76]:
revdict(vv) = [k for (k,v) in DataLoad.UDEPREL if v==vv][1]

revdict (generic function with 1 method)

In [165]:
[k for (k,v) in DataLoad.UDEPREL if v==2][1]

"acl"

In [251]:
modelpos(datmb2_test[156][1], TestMode())[1]

26×25×1 Array{Float64,3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  1.0  1.0  1.0  1.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  1.0  1.0  1.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  1.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  

In [351]:
my_accuracy_rel_parsed(modelpos,datmb2_test)

0.861577080926007

In [53]:
function minibRELPOS_test(data4c,batchsize)
    #datdat = sort(data4c, by=x->length(x[1][1]), rev=true)
    datdat = data4c
    lentsdatdat = lents2(datdat)
    uniqdat = unique(lentsdatdat)
    lennnum = [(i,count(x->x==i,lentsdatdat),j) for (j,i) in enumerate(uniqdat)]
    i=1
    j=0
    k=0
    batdat = []
    for (a,b,c) in lennnum
        bb = b
        j += b
        j = i+b
        k=j
        while bb >= batchsize
            j =i+batchsize-1

            push!(batdat, (  (  cat([x[1][1] for x in datdat[i:j]]...,dims=1) 
                        , cat([x[1][2] for x in datdat[i:j]]...,dims=1)  
                        , cat([x[1][3] for x in datdat[i:j]]...,dims=1)            )
                  push!(arcs, 0)      
                    , (cat([x[2][1] for x in datdat[i:j]]...,dims=2), Array{Int64,2}(cat([x[2][2] for x in datdat[i:j]]...,dims=2)))))
            i=j+1
            bb -= batchsize
        end
        i += bb
    end
    batdat
end

minibRELPOS_test (generic function with 1 method)

In [173]:
open("myfile.txt", "w") do io
       write(io, "Hello world!
LELELELELE")
   end;

In [270]:
S = modelpos(datmb2_test[1][1])[1]

8×7×1 KnetArray{Float32,3}:
[:, :, 1] =
  -2.6366    -28.9899  -25.0186   -6.64678  -44.2552  -23.7597   -23.1713
  -5.26424   -16.0953  -12.6854   -3.25331  -17.2341   -5.82506  -11.029 
 -16.4196    -31.2665  -12.7534  -36.1303   -48.1432  -21.6373   -25.7826
 -14.3727    -19.0423  -13.4384  -33.4157   -25.277   -24.1869   -29.4958
  -0.968476   24.0427   16.6474  -26.2328   -19.0048   10.9088   -14.8357
 -24.9932    -55.7365  -28.9641  -54.1607   -29.0197  -25.5496   -35.5657
 -16.0726    -24.0717  -20.99    -38.6905    13.487   -15.638    -23.2397
 -39.1114    -85.6197  -59.6254  -63.041    -59.301   -49.7375   -57.3334

In [326]:
Svalr = reshape(value(S), 8,7) #2d version , assert(bbs==1)
Svalrcpu = Array(Svalr)
S_cpu_cat = hcat(Array(zeros(8,1)), Svalrcpu)
[S_cpu_cat[i,i] = -999999 for i in 1:size(S_cpu_cat,1)]
rel_indices = [x+1 for     push!(arcs, 0)x in edmonds(S_cpu_cat)]

7-element Array{Int64,1}:
 1
 5
 5
 5
 7
 5
 8

In [309]:
S_cpu_cat[5,:]

8-element Array{Float64,1}:
       0.0              
      -0.968476414680481
      24.042692184448242
      16.647415161132812
 -999999.0              
     -19.004844665527344
      10.908795356750488
     -14.835738182067871

In [343]:
edmonds(S_cpu_cat)

7-element Array{Int64,1}:
 0
 2
 3
 4
 5
 6
 7

In [304]:
cky(S_cpu_cat)

7-element Array{Int64,1}:
 0
 4
 4
 1
 6
 4
 0

In [78]:
using PyCall
chu = pyimport("chuliu_edmonds2")


PyObject <module 'chuliu_edmonds2' from '/home/sfdgb/dlprj2/aws_oldfiles/chuliu_edmonds2.py'>

In [374]:
chu.chuliu_edmonds(S_cpu_cat')[2:end]

7-element Array{Int64,1}:
 0
 4
 4
 1
 6
 4
 1